#### 시간별 데이터 수집

In [1]:
import json
import datetime
import requests
import pandas as pd

# 팍스넷에서 데이터 주어옴.

# stock_no = "226490"
# file_name = "data_kodex_kospi"
# rename_prefix = "KDX"

stock_no = "114800"
file_name = "data_kodex_kospi_inverse"
rename_prefix = "KDI"

url = "http://www.paxnet.co.kr/stock/analysis/chart?abbrSymbol={}".format(stock_no)
res = requests.get(url)
result = res.text.split("var sise60minJson = \'")[1].split("\';\r\n \t")[0]
json_res = json.loads(result)
df_src = pd.DataFrame(json_res["body"]["chartDataList"])

In [2]:
df = df_src.copy()
date_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[0] for x in df["tradeDt"]]
time_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[1].split(":")[0] for x in df["tradeDt"]]
df["date"] = date_list
df["time"] = time_list
df["datetime"] = df["date"].str.replace("-", "") + df["time"]
df = df[['date', 'time', 'datetime', 'openPrice', 'closePrice', 'highPrice', 'lowPrice', 'volume']]
df.rename(columns={
    "date" : "DATE",
    "time" : "TIME",
    "datetime" : "DATETIME",    
    "openPrice" : "{}_OPN".format(rename_prefix),
    "closePrice" : "{}_CLS".format(rename_prefix),
    "highPrice" : "{}_HIGH".format(rename_prefix),
    "lowPrice" : "{}_LOW".format(rename_prefix),
    "volume" : "{}_VOL".format(rename_prefix)
}, inplace=True)

df_old = pd.read_excel("{}.xlsx".format(file_name))
df_new = pd.concat([df_old, df], axis=0, ignore_index=True)
df_new["DATETIME_INT"] = df_new["DATETIME"].astype("int")
df_new.drop_duplicates(["DATETIME_INT"], inplace=True)
df_new.drop(columns=["DATETIME_INT"], inplace=True)
df_new.reset_index(drop=True, inplace=True)
df_new.to_excel("{}.xlsx".format(file_name), index=False)


#### 일자별 데이터 수집

In [4]:
import json
import datetime
import requests
import pandas as pd

# stock_no = "226490"
# file_name = "data_kodex_kospi_by_day"
# rename_prefix = "KDX"

stock_no = "114800"
file_name = "data_kodex_kospi_inverse_by_day"
rename_prefix = "KDI"

url = "http://www.paxnet.co.kr/stock/analysis/chart?abbrSymbol={}".format(stock_no)
res = requests.get(url)
result = res.text.split("var siseDayJson = \'")[1].split("\';\r\n \t")[0]
json_res = json.loads(result)
df_src = pd.DataFrame(json_res["body"]["chartDataList"])

df = df_src.copy()
date_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[0] for x in df["tradeDt"]]
time_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[1].split(":")[0] for x in df["tradeDt"]]
df["date"] = date_list
df["time"] = time_list
df["datetime"] = df["date"].str.replace("-", "") + df["time"]
df = df[['date', 'time', 'datetime', 'openPrice', 'closePrice', 'highPrice', 'lowPrice', 'volume']]
df.rename(columns={
    "date" : "DATE",
    "time" : "TIME",
    "datetime" : "DATETIME",    
    "openPrice" : "{}_OPN".format(rename_prefix),
    "closePrice" : "{}_CLS".format(rename_prefix),
    "highPrice" : "{}_HIGH".format(rename_prefix),
    "lowPrice" : "{}_LOW".format(rename_prefix),
    "volume" : "{}_VOL".format(rename_prefix)
}, inplace=True)

try:
    df_old = pd.read_excel("{}.xlsx".format(file_name))
except:
    df_new = df.copy()
else:
    df_new = pd.concat([df_old, df], axis=0, ignore_index=True)

df_new["DATETIME_INT"] = df_new["DATETIME"].astype("int")
df_new.drop_duplicates(["DATETIME_INT"], inplace=True)
df_new.drop(columns=["DATETIME_INT"], inplace=True)
df_new.reset_index(drop=True, inplace=True)
df_new.to_excel("{}.xlsx".format(file_name), index=False)
df_new

,DATE,TIME,DATETIME,KDI_OPN,KDI_CLS,KDI_HIGH,KDI_LOW,KDI_VOL
0,2018-05-18,18,2018051818,6145,6135,6165,6135,5206379.0
1,2018-05-21,18,2018052118,6130,6135,6185,6115,5707844.0
2,2018-05-23,18,2018052318,6140,6090,6145,6090,5391745.0
3,2018-05-24,18,2018052418,6075,6115,6130,6075,3346092.0
4,2018-05-25,18,2018052518,6145,6095,6145,6090,5608692.0
...,...,...,...,...,...,...,...,...
995,2022-06-03,18,2022060318,4405,4420,4430,4395,14872251.0
996,2022-06-07,18,2022060718,4440,4500,4510,4440,24379235.0
997,2022-06-08,18,2022060818,4490,4490,4505,4465,16861667.0
998,2022-06-09,18,2022060918,4505,4495,4525,4485,19349290.0
